# Flop Shovers Explorer

Examine first-in flop shoves (jam as the initial bet) to understand what holdings reach for maximum leverage. This notebook mirrors the formatting of the existing flop explorers and reuses the same hand-type groupings for easy comparison.

## Questions To Answer

- What combos show up when someone shoves first-in on the flop?
- How do those holdings shift between short-stack (<30bb) and deeper-stack (≥30bb) situations?
- Which primary hand groups (Air, Weak Pair, Top Pair, etc.) dominate these jams?

In [1]:
from pathlib import Path
import os
import sqlite3


def _locate_project_root() -> Path:
    current = Path().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "AGENTS.md").exists():
            return candidate
    raise FileNotFoundError("Repository root not found from notebook location.")


PROJECT_ROOT = _locate_project_root()
del _locate_project_root

DB_CANDIDATES = [
    PROJECT_ROOT / "drivehud" / "drivehud.db",
    PROJECT_ROOT / "data" / "warehouse" / "drivehud.sqlite",
    PROJECT_ROOT / "data" / "warehouse" / "ignition.sqlite",
]


def _has_hand_histories(db_path: Path) -> bool:
    try:
        with sqlite3.connect(db_path) as conn:
            cur = conn.execute(
                "SELECT 1 FROM sqlite_master WHERE type='table' AND name='HandHistories'"
            )
            return cur.fetchone() is not None
    except sqlite3.Error:
        return False


for candidate in DB_CANDIDATES:
    if candidate.exists() and _has_hand_histories(candidate):
        DB_PATH = candidate
        break
else:
    checked = os.linesep.join(str(p) for p in DB_CANDIDATES)
    raise FileNotFoundError(
        "Database with HandHistories not found. Checked:" + os.linesep + checked
    )

CACHE_DIR = PROJECT_ROOT / "analysis" / "cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

del _has_hand_histories

In [2]:
import sys
import json
import sqlite3
import xml.etree.ElementTree as ET
from typing import Dict, List, Sequence

import numpy as np
import pandas as pd
from IPython.display import display

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from analysis.sqlite_utils import connect_readonly
from analysis.cbet_utils import (
    BET_TYPES,
    RAISE_TYPES,
    classify_hand,
    extract_big_blind,
    parse_cards_text,
)

In [3]:
# --- Configuration ---
FORCE_RELOAD = False
STACK_THRESHOLD_BB = 30

CACHE_PATH = CACHE_DIR / "flop_shovers.json"

In [4]:
PRIMARY_GROUPS = {
    "Air": ["Air"],
    "Weak Pair": ["Underpair", "Bottom Pair", "Middle Pair"],
    "Top Pair": ["Top Pair"],
    "Overpair": ["Overpair"],
    "Two Pair": ["Two Pair"],
    "Trips/Set": ["Trips/Set"],
    "Monster": ["Straight", "Flush", "Full House", "Quads"],
    "Draw": [],
}

PRIMARY_ORDER = [
    "Monster",
    "Trips/Set",
    "Two Pair",
    "Overpair",
    "Top Pair",
    "Weak Pair",
    "Draw",
    "Air",
    "Other",
]

_PRIMARY_LOOKUP = {
    primary: group
    for group, members in PRIMARY_GROUPS.items()
    for primary in members
}

AGGRESSIVE_TYPES = BET_TYPES.union(RAISE_TYPES)


def determine_primary_group(primary: str | None, classification: Dict[str, object]) -> str:
    if classification.get("flush_draw") or classification.get("oesd_dg"):
        return "Draw"
    if primary:
        mapped = _PRIMARY_LOOKUP.get(primary)
        if mapped:
            return mapped
    return "Other"


def _player_stacks(root: ET.Element) -> Dict[str, float]:
    stacks: Dict[str, float] = {}
    for player_node in root.findall('.//game/general/players/player'):
        player = player_node.attrib.get("name")
        if not player:
            continue
        try:
            chips = float(player_node.attrib.get("chips") or 0.0)
        except (TypeError, ValueError):
            chips = 0.0
        stacks[player] = chips
    return stacks


def _ensure_primary_groups(events: List[Dict]) -> bool:
    updated = False
    for event in events:
        desired = determine_primary_group(event.get("primary"), event)
        if event.get("primary_group") != desired:
            event["primary_group"] = desired
            updated = True
    return updated


def load_flop_shove_events(
    db_path: Path,
    cache_path: Path | None = None,
    force: bool = False,
) -> List[Dict]:
    if cache_path and cache_path.exists() and not force:
        with cache_path.open("r", encoding="utf-8") as fh:
            cached = json.load(fh)
        if cached:
            if _ensure_primary_groups(cached) and cache_path:
                cache_path.parent.mkdir(parents=True, exist_ok=True)
                with cache_path.open("w", encoding="utf-8") as out:
                    json.dump(cached, out, ensure_ascii=False, indent=2)
        return cached

    events: List[Dict] = []
    with connect_readonly(db_path) as conn:
        conn.row_factory = sqlite3.Row
        cur = conn.cursor()
        cur.execute("SELECT HandNumber, HandHistory FROM HandHistories")
        for row in cur:
            hand_number = row["HandNumber"]
            try:
                root = ET.fromstring(row["HandHistory"])
            except ET.ParseError:
                continue

            big_blind = extract_big_blind(root)
            if not big_blind or big_blind <= 0:
                continue

            pocket_cards: Dict[str, Sequence[tuple[str, int, str]]] = {}
            for node in root.findall('.//round[@no="1"]/cards'):
                player = node.attrib.get("player")
                cards = parse_cards_text(node.text)
                if player and len(cards) == 2:
                    pocket_cards[player] = cards
            if not pocket_cards:
                continue

            flop_cards = None
            for card_node in root.findall('.//round[@no="2"]/cards'):
                if card_node.attrib.get("type") == "Flop":
                    flop_cards = parse_cards_text(card_node.text)
                    break
            if not flop_cards or len(flop_cards) != 3:
                continue

            stacks = _player_stacks(root)
            if not stacks:
                continue

            hero_name = next((name for name in stacks if name.lower() == "hero"), None)

            rounds = sorted(
                root.findall('.//round'), key=lambda r: int(r.attrib.get('no', '0'))
            )
            first_bet_handled = False

            for rnd in rounds:
                round_no = int(rnd.attrib.get('no', '0'))

                for action in rnd.findall('action'):
                    player = action.attrib.get("player")
                    if not player:
                        continue
                    act_type = action.attrib.get("type")
                    try:
                        amount = float(action.attrib.get("sum") or 0.0)
                    except (TypeError, ValueError):
                        amount = 0.0

                    stack_before = stacks.get(player, 0.0)

                    if (
                        round_no == 2
                        and not first_bet_handled
                        and act_type in AGGRESSIVE_TYPES
                        and amount > 0
                    ):
                        first_bet_handled = True
                        if act_type == "7" and player in pocket_cards:
                            classification = classify_hand(pocket_cards[player], flop_cards)
                            group = determine_primary_group(classification.get("primary"), classification)
                            events.append(
                                {
                                    "hand_number": hand_number,
                                    "player": player,
                                    "is_hero": bool(hero_name and player == hero_name),
                                    "stack_before": stack_before,
                                    "stack_bb": stack_before / big_blind,
                                    "bet_amount": amount,
                                    "bet_bb": amount / big_blind,
                                    "primary": classification.get("primary"),
                                    "primary_group": group,
                                    "has_flush_draw": bool(classification.get("flush_draw")),
                                    "has_oesd_dg": bool(classification.get("oesd_dg")),
                                    "made_flush": bool(classification.get("made_flush")),
                                    "made_straight": bool(classification.get("made_straight")),
                                    "made_full_house": bool(classification.get("made_full")),
                                    "hole_cards": " ".join(card for _, _, card in pocket_cards[player]),
                                    "flop_cards": " ".join(card for _, _, card in flop_cards),
                                    "big_blind": big_blind,
                                }
                            )
                        break

                    if amount > 0:
                        stacks[player] = stack_before - amount

                if first_bet_handled:
                    break

    _ensure_primary_groups(events)
    if cache_path:
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        with cache_path.open("w", encoding="utf-8") as fh:
            json.dump(events, fh, ensure_ascii=False, indent=2)

    return events


def summarise_by_group(frame: pd.DataFrame) -> pd.DataFrame:
    if frame.empty:
        return pd.DataFrame(columns=["stack_bucket", "group", "events", "pct"])

    summaries = []
    for bucket, bucket_df in frame.groupby("stack_bucket"):
        total = len(bucket_df)
        for group in PRIMARY_ORDER:
            count = int((bucket_df["primary_group"] == group).sum())
            pct = (count / total * 100.0) if total else 0.0
            summaries.append(
                {
                    "stack_bucket": bucket,
                    "group": group,
                    "events": count,
                    "pct": pct,
                }
            )
    result = pd.DataFrame(summaries)
    if not result.empty:
        result.sort_values(["stack_bucket", "pct"], ascending=[True, False], inplace=True)
    return result


def style_pct_table(df: pd.DataFrame, title: str | None = None):
    if df.empty:
        return df.style.hide(axis='index')
    subset = df[["group", "events", "pct"]].copy()
    styler = (
        subset
        .style.hide(axis='index')
        .format({
            "events": "{:,.0f}".format,
            "pct": "{:.1f}%".format,
        })
        .background_gradient(cmap="Blues", subset=["pct"])
    )
    if title:
        styler = styler.set_caption(title)
    return styler


In [5]:
from analysis.flop_texture_tables import (
    PRIMARY_DISPLAY_ORDER,
    TEXTURE_ORDER,
    COMBINED_TEXTURE_ORDER,
    prepare_texture_dataframe,
    compute_primary_texture_table,
    compute_combined_texture_table,
    compute_percent_table,
    style_heatmap_table,
    style_heatmap_percentage_table,
)


In [6]:
events = load_flop_shove_events(
    DB_PATH,
    cache_path=CACHE_PATH,
    force=FORCE_RELOAD,
)
events_df = pd.DataFrame(events)

In [7]:

if events_df.empty:
    print("No flop shove events with known hole cards were found.")
else:
    working_df = events_df.copy()
    working_df["is_hero"] = working_df["is_hero"].fillna(False)

    population_df = working_df[~working_df["is_hero"]].copy()
    if population_df.empty:
        print("No non-hero flop shove events available for analysis.")
    else:
        population_df["stack_bb"] = population_df["stack_bb"].astype(float)
        population_df["stack_bucket"] = np.where(
            population_df["stack_bb"] >= STACK_THRESHOLD_BB,
            ">=30bb",
            "<30bb",
        )
        if "primary_group" not in population_df.columns:
            population_df["primary_group"] = population_df.apply(
                lambda row: determine_primary_group(row.get("primary"), row),
                axis=1,
            )
        else:
            population_df["primary_group"] = population_df["primary_group"].fillna("Other")

        population_df["texture_bucket"] = population_df["flop_cards"].apply(derive_texture)

        bucket_counts = (
            population_df.groupby("stack_bucket")["hand_number"].count()
            .rename("events")
            .reset_index()
            .sort_values("stack_bucket")
        )
        if not bucket_counts.empty:
            display(
                bucket_counts
                .style.hide(axis='index')
                .format({"events": "{:,.0f}".format})
                .set_caption("Flop shove counts by stack bucket (non-hero)")
            )

        group_summary = summarise_by_group(population_df)
        short_table = group_summary[group_summary["stack_bucket"] == "<30bb"].copy()
        deep_table = group_summary[group_summary["stack_bucket"] == ">=30bb"].copy()

        if not short_table.empty:
            display(
                style_pct_table(
                    short_table,
                    title="<30bb: Primary hand-group distribution (% of shoves)",
                )
            )
        else:
            print("No <30bb shoves recorded (non-hero).")

        if not deep_table.empty:
            display(
                style_pct_table(
                    deep_table,
                    title=">=30bb: Primary hand-group distribution (% of shoves)",
                )
            )
        else:
            print("No >=30bb shoves recorded (non-hero).")

        short_df = population_df[population_df["stack_bucket"] == "<30bb"].copy()
        deep_df = population_df[population_df["stack_bucket"] == ">=30bb"].copy()

        for label, dataset in [("<30bb", short_df), (">=30bb", deep_df)]:
            if dataset.empty:
                print(f"No {label} shoves recorded for texture analysis.")
                continue
            primary_df = prepare_texture_dataframe(dataset)
            primary_table = compute_primary_texture_table(primary_df)
            display(
                style_heatmap_table(
                    primary_table,
                    f"{label}: Suited/paired textures vs primary group",
                )
            )
            primary_percent_table = compute_percent_table(primary_table)
            display(
                style_heatmap_percentage_table(
                    primary_percent_table,
                    f"{label}: Suited/paired textures (% of column)",
                )
            )

            combined_table = compute_combined_texture_table(primary_df)
            display(
                style_heatmap_table(
                    combined_table,
                    f"{label}: Connected/high/low textures vs primary group",
                )
            )
            combined_percent_table = compute_percent_table(combined_table)
            display(
                style_heatmap_percentage_table(
                    combined_percent_table,
                    f"{label}: Connected/high/low textures (% of column)",
                )
            )


stack_bucket,events
<30bb,122
>=30bb,116


group,events,pct
Air,51,41.8%
Weak Pair,32,26.2%
Top Pair,17,13.9%
Two Pair,16,13.1%
Overpair,3,2.5%
Trips/Set,2,1.6%
Monster,1,0.8%
Draw,0,0.0%
Other,0,0.0%


group,events,pct
Air,39,33.6%
Top Pair,26,22.4%
Two Pair,15,12.9%
Weak Pair,15,12.9%
Overpair,13,11.2%
Monster,5,4.3%
Trips/Set,3,2.6%
Draw,0,0.0%
Other,0,0.0%


,Paired Monotone,Paired Two-tone,Paired Rainbow,Unpaired Monotone,Unpaired Two-tone,Unpaired Rainbow,Total (count),Total (pct)
Air,0,5,4,1,28,13,51,41.8%
Draw,0,0,0,0,0,0,0,0.0%
Weak Pair,0,0,0,6,22,4,32,26.2%
Top Pair,0,0,0,0,9,8,17,13.9%
Overpair,0,0,0,0,1,2,3,2.5%
Two Pair,0,5,9,0,1,1,16,13.1%
Trips/Set,0,0,1,0,1,0,2,1.6%
Monster,0,0,1,0,0,0,1,0.8%
Total (count),0,10,15,7,62,28,122,100.0%
Total (pct),0.0%,8.2%,12.3%,5.7%,50.8%,23.0%,122,100.0%


,Paired Monotone,Paired Two-tone,Paired Rainbow,Unpaired Monotone,Unpaired Two-tone,Unpaired Rainbow,Total (count),Total (pct)
Air,0,8,2,3,17,9,39,33.6%
Draw,0,0,0,0,0,0,0,0.0%
Weak Pair,0,0,0,2,10,3,15,12.9%
Top Pair,0,0,0,3,15,8,26,22.4%
Overpair,0,0,0,0,9,4,13,11.2%
Two Pair,0,5,4,1,2,3,15,12.9%
Trips/Set,0,0,1,0,2,0,3,2.6%
Monster,0,0,2,1,2,0,5,4.3%
Total (count),0,13,9,10,57,27,116,100.0%
Total (pct),0.0%,11.2%,7.8%,8.6%,49.1%,23.3%,116,100.0%


## Next Analysis Ideas

- Break the distributions out by board texture (paired, monotone, high-card) to see when jamming ranges shift.
- Compare villain response frequencies (fold/call/raise) after these shoves to identify profitable bluff windows.
- Add stake/time filters and integrate with `v_hand_bb` so the UI can surface heads-up vs multiway flop jams separately.